# AITom Tutorial

# 1.1 Particle picking

In [ ]:
import aitom.io.file as io_file
import aitom.image.vol.util as im_vol_util
import os, sys, copy, uuid, shutil
from aitom.bin.picking import picking
# file path
path = '/ldap_shared/home/v_zhenxi_zhu/data/aitom_demo_single_particle_tomogram.mrc'
output_dir = './tmp/picking'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)
# select sigma1 automatically 
mrc_header = io_file.read_mrc_header(path)
voxel_spacing_in_nm = mrc_header['MRC']['xlen'] / mrc_header['MRC']['nx'] / 10
sigma1 = max(int(7 / voxel_spacing_in_nm), 2)  # In general, 7 is optimal sigma1 val in nm according to the paper and sigma1 should at least be 2
# or select it manually
if True:
    sigma1 = 5
result = picking(path, s1=sigma1, s2=sigma1*1.1, t=3, find_maxima=False, partition_op=None, multiprocessing_process_num=10, pick_num=1000)
print("DoG done, %d particles picked" % len(result))

# Save subvolumes of peaks for autoencoder input
dump_subvols = True
if dump_subvols: # use later for autoencoder
    from aitom.classify.deep.unsupervised.autoencoder.autoencoder_util import peaks_to_subvolumes
    subvols_loc = os.path.join(output_dir,"demo_single_particle_subvolumes.pickle")
    a = io_file.read_mrc_data(path)
    d = peaks_to_subvolumes(im_vol_util.cub_img(a)['vt'], result, 32)
    io_file.pickle_dump(d, subvols_loc)
    print("Save subvolumes .pickle file to:", subvols_loc)


# 1.2 Visualization of particle picking result

In [ ]:
from aitom.filter.gaussian import smooth
import matplotlib.pyplot as plt
import numpy as np

# d = {v_siz:(32,32,32), vs:{uuid0:{center, v, id}, uuid1:{center, v, id} ... }}
import aitom.io.file as io_file
subvols_loc = './tmp/picking/demo_single_particle_subvolumes.pickle'
d = io_file.pickle_load(subvols_loc)
path = '/ldap_shared/home/v_zhenxi_zhu/data/aitom_demo_single_particle_tomogram.mrc'
a = io_file.read_mrc_data(path)

centers = []
uuids = []
# for v in d['vs'].values():
for k,v in d['vs'].items():
    if v['v'] is not None:
        centers.append(v['center'])
        uuids.append(k)

# denoise
if True:
    a_smooth = smooth(a,2)
else:
    a_smooth = a

# select radius of circles and slice number to visualize
R ,slice_num = 10,40  

centers = np.array(centers)

slice_centers = centers[(centers[:,2]-slice_num)**2<R**2]
img = a_smooth[:,:,slice_num]
plt.rcParams['figure.figsize'] = (15.0, 12.0)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.axis('off')
for center_num in range(len(slice_centers)):
    y, x = slice_centers[center_num][0:2]
    r = np.sqrt(R**2 - (slice_centers[center_num][2]-slice_num)**2)
    circle = plt.Circle((x, y), r, color='b', fill=False)
    plt.gcf().gca().add_artist(circle)
ax_u = ax.imshow(img, cmap = 'gray')
print('Visualization of the whole tomogram:')


In [ ]:
print('Visualization of the subtomogram:')
subvol_num = 113
y, x, z = centers[subvol_num]
img = a_smooth[:,:,z]
plt.rcParams['figure.figsize'] = (10.0, 8.0)
fig = plt.figure()
ax = fig.add_subplot(111)
circle = plt.Circle((x, y), R, color='b', fill=False)
plt.gcf().gca().add_artist(circle)
plt.axis('off')
print('%d of %d, uuid = %s' %(subvol_num,len(centers),uuids[subvol_num]))
ax_u = ax.imshow(img, cmap = 'gray')

# 1.3 Manual selection

In [ ]:
# (Optional) manual remove particles
remove_particles = [0,4,9,11,18,24,26,30,34,35,38,53,66,78,96,99,111,112,118,123,124]
if True:
    particles_num = 100  # the number of selected particles
else:
    particles_num = len(centers) - len(remove_particles)  # select all
import aitom.io.file as AIF
result = {}
result['v_siz'] = d['v_siz']
result['vs'] = {}
remove_particles = np.array(remove_particles)
# d = {v_siz:(32,32,32), vs:{uuid0:{center, v, id}, uuid1:{center, v, id} ... }}

for i in range(len(centers)):
    if i in remove_particles:
        continue
    uuid_i = uuids[i]
    result['vs'][uuid_i] = d['vs'][uuid_i]
    if len(result['vs']) >= particles_num:
        break
assert len(result['vs']) == particles_num
subvols_loc = './tmp/picking/selected_demo_single_particle_subvolumes.pickle'
AIF.pickle_dump(result, subvols_loc)
print("Save subvolumes .pickle file to:", subvols_loc)
    

# 2.1 Autoencoder (single particle)

In [ ]:
import os,shutil
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
subvols_loc = './tmp/picking/selected_demo_single_particle_subvolumes.pickle'
output_dir = './tmp/autoencoder_single_particle'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

import aitom.classify.deep.unsupervised.autoencoder.autoencoder as AE
import time
s_time = time.time()

out_dir = output_dir
single_particle_param = [subvols_loc, 'None', "False", 1]  # number of clusters = 1

parameters_demo = single_particle_param # choose one of the above 
import aitom.io.file as AIF
d = AIF.pickle_load(parameters_demo[0]) # pickle data file of CECT small subvolumes

img_org_file = parameters_demo[1]#A tomogram file in .rec format, which can be None when pose normalization is not required
pose = eval(parameters_demo[2])#Whether the optional pose normalization step should be applied  True or False
clus_num = int(parameters_demo[3])# The number of clusters

AE.encoder_simple_conv_test(d=d, pose=pose, img_org_file=img_org_file, out_dir=out_dir, clus_num=clus_num)
AE.kmeans_centers_plot(AE.op_join(out_dir, 'clus-center'))


## Visualization of cluster centers

In [ ]:
from matplotlib.image import imread
fig_dir = os.path.join(out_dir,'clus-center/fig')
for filename in os.listdir(fig_dir):
    img = imread(os.path.join(fig_dir,filename))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.axis('off')
    ax_u = ax.imshow(img, cmap = 'gray')


# 2.2 Autoencoder (multi particles)

In [ ]:
import pickle,os,shutil,uuid
import aitom.io.file as AIF
import numpy as N
pickle_path = '/ldap_shared/home/v_zhenxi_zhu/classification_and_averaging/aitom_demo_subtomograms.pickle'  # demo subtomograms
subvols_loc = './tmp/picking/selected_demo_single_particle_subvolumes.pickle'
pickle_data = AIF.pickle_load(pickle_path)
d = AIF.pickle_load(subvols_loc)
subvols = []
for v in d['vs'].values():
    if v['v'] is not None:
        subvols.append(v['v'])
# subtom = pickle_data['5T2C_data'] + pickle_data['1KP8_data'] + subvols[:100]
subtom = pickle_data['1KP8_data'] + pickle_data['1KP8_data']
print('Total subtomograms: ',len(subtom))
subvols_loc = './tmp/picking/subvolumes.pickle'
d = {}
d['v_siz'] = N.array([32,32,32])
d['vs'] = {}
labels = {}
for i in range(len(subtom)):
    uuid_i = str(uuid.uuid4())
    d['vs'][uuid_i] = {}
    d['vs'][uuid_i]['center'] = None
    d['vs'][uuid_i]['id'] = uuid_i
    d['vs'][uuid_i]['v'] = subtom[i]
    d['vs'][uuid_i]['label'] = int(i/100)

AIF.pickle_dump(d, subvols_loc)
print("Save subvolumes .pickle file to:", subvols_loc)

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
subvols_loc = './tmp/picking/subvolumes.pickle'
output_dir = './tmp/autoencoder_multi_particles'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

import aitom.classify.deep.unsupervised.autoencoder.autoencoder as AE
multi_particle_param = [subvols_loc, 'None', "False", 2]  # number of clusters
parameters_demo = multi_particle_param

import aitom.io.file as AIF
d = AIF.pickle_load(parameters_demo[0]) # pickle data file of CECT small subvolumes
img_org_file = parameters_demo[1]#A tomogram file in .rec format, which can be None when pose normalization is not required
pose = eval(parameters_demo[2])#Whether the optional pose normalization step should be applied  True or False
clus_num = int(parameters_demo[3])# The number of clusters
out_dir = output_dir

AE.encoder_simple_conv_test(d=d, pose=pose, img_org_file=img_org_file, out_dir=out_dir, clus_num=clus_num)
AE.kmeans_centers_plot(AE.op_join(out_dir, 'clus-center'))


## Visualization of cluster centers

In [ ]:
from matplotlib.image import imread

import os
import matplotlib.pyplot as plt
out_dir = './tmp/autoencoder_multi_particles'

fig_dir = os.path.join(out_dir,'clus-center/fig')
for filename in os.listdir(fig_dir):
    img = imread(os.path.join(fig_dir,filename))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.axis('off')
    ax_u = ax.imshow(img, cmap = 'gray')